# Exercise 1

### (a) 

When a client (Bob) receives a public-key certificate from a web server (Alice), the accepted
procedure is the following:

1. **Check the server identity**  
   The client verifies that the identity contained in the certificate is correct for the
   intended server (for example, the DNS name of the web server).

2. **Check the validity period**  
   The client verifies that the current time is within the certificate’s validity interval
   (start and end dates).

3. **Check certificate meta-information**  
   The client verifies that the certificate’s meta-information and extensions make sense for
   the intended application (for example, key usage, basic constraints).  
   If a critical extension is present and not recognized, the certificate must be rejected.

4. **Verify the CA’s signature**  
   The client obtains the public key of the issuing Certification Authority (CA) and verifies
   the digital signature on the certificate.

5. **Establish trust in the CA (certificate chain validation)**  
   If the CA is not directly trusted, the client iteratively validates the certificate chain
   until reaching a trusted root CA that is implicitly trusted (pre-installed in the system).

6. **Accept or reject the certificate**  
   If all checks succeed, the client accepts the certificate and uses the server’s public key.
   Otherwise, the certificate is rejected.


### (b) 

**Certificate revocation** is the process of invalidating a public-key certificate before the
end of its validity period. Certificates that are outside their validity dates are automatically
invalid, but revocation is required when a certificate must be considered invalid while it still
appears valid.

#### Reasons for certificate revocation
A certificate may need to be revoked when:
- The secret (private) key associated with the certificate is lost or compromised.
- There is a data breach affecting the secrecy of the key.
- The information contained in the certificate (identity or meta-data) becomes incorrect.
- The certificate can no longer be trusted for its intended purpose.

#### Revocation process
1. **Revocation by the CA**  
   The Certification Authority (CA) formally revokes the certificate and records it as invalid.

2. **Publication of revocation information**  
   The CA periodically publishes a **Certificate Revocation List (CRL)**, which is a blacklist
   of revoked certificates. Exceptional CRLs may also be published on a best-effort basis.

3. **Distribution of revocation data**  
   Certificate extensions typically include URLs indicating where CRLs can be obtained.

4. **Revocation checking by clients**  
   Clients are expected to obtain and check the most recent revocation information before
   accepting a certificate, although traditional client support for CRLs is limited.

#### Alternative revocation mechanisms
- **Trusted Service Provider Lists (TSL):** up-to-date whitelists of trusted certificates,
  typically used in closed or high-security environments.
- **Online Certificate Status Protocol (OCSP):** a trusted online service that checks
  revocation status on behalf of the client.
- **Certificate pinning:** applications or browsers maintain their own whitelist of
  acceptable certificates for specific entities.

If revocation information indicates that a certificate is revoked, the client must reject it.



# Exercise 2

In [ ]:
# For this exercise, we need to use SageMath
n = 620496404349687915307910174617
c = 358624662650643040547102063483144791182626860568435345308004

p, q = factor(n)[0][0], factor(n)[1][0]
print("p =", p)
print("q =", q)

# Compute lambda = lcm(p-1, q-1)
lam = lcm(p-1, q-1)
print("lambda =", lam)

# Standard Paillier choice
g = n + 1

# L function
def L(u):
    return (u - 1) // n

# Compute mu
nsq = n * n
g_lam = pow(g, lam, nsq)
mu = inverse_mod(L(g_lam), n)
print("mu =", mu)

# Decrypt ciphertext
c_lam = pow(c, lam, nsq)
m = (L(c_lam) * mu) % n
print("Decrypted message m =", m)

p = 772886493240161
q = 802829923639097
lambda = 77562050543710792448936661920
mu = 148596065928223072848222089179
Decrypted message m = 455667


# Exercise 3. Paillier Voting System Implementation

The Paillier cryptosystem procedures as requested:

* **`genPrivate(sz)`**: Generates a public key $n$ and private components $p, q$ where $n = p \cdot q$ and $p, q$ are primes of size sz.
* **`voteYes(fileName, n)`**: Encrypts a vote of $1$ and appends it to a file.
* **`voteNo(fileName, n)`**: Encrypts a vote of $0$ and appends it to a file.
* **`getResults(fileName, n, phi)`**: Aggregates the encrypted votes and decrypts the total using Euler's totient, `phi`.

In [5]:
import random
import os
import math

def genPrivate(sz):
    primes = []
    while len(primes) < 2:
        p = random.getrandbits(sz)
        p |= (1 << (sz - 1)) | 1
        if p == 2 or p == 3:
            primes.append(p)
            continue
        if p < 2 or p % 2 == 0:
            continue
        d = p - 1
        r = 0
        while d % 2 == 0:
            d //= 2
            r += 1
        is_prime = True
        for _ in range(40):
            a = random.randrange(2, p - 1)
            x = pow(a, d, p)
            if x == 1 or x == p - 1:
                continue
            for _ in range(r - 1):
                x = pow(x, 2, p)
                if x == p - 1:
                    break
            else:
                is_prime = False
                break
        if is_prime:
            if len(primes) == 0 or primes[0] != p:
                primes.append(p)
    n = primes[0] * primes[1]
    return (n, primes[0], primes[1])

def voteYes(fileName, n):
    n_sq = n * n
    while True:
        r = random.randrange(1, n)
        if math.gcd(r, n) == 1:
            break
    c = ((1 + n) * pow(r, n, n_sq)) % n_sq
    with open(fileName, "a") as f:
        f.write(str(c) + "\n")

def voteNo(fileName, n):
    n_sq = n * n
    while True:
        r = random.randrange(1, n)
        if math.gcd(r, n) == 1:
            break
    c = pow(r, n, n_sq)
    with open(fileName, "a") as f:
        f.write(str(c) + "\n")

def getResults(fileName, n, phi):
    if not os.path.exists(fileName):
        return 0
    n_sq = n * n
    c_product = 1
    with open(fileName, "r") as f:
        for line in f:
            if line.strip():
                c_product = (c_product * int(line.strip())) % n_sq
    c_phi = pow(c_product, phi, n_sq)
    L_u = (c_phi - 1) // n
    phi_inv = pow(phi, -1, n)
    result = (L_u * phi_inv) % n
    print(result)
    return result
if __name__ == "__main__":
    print("--- Initializing Election System ---")
    
    sz = 128
    print(f"Generating {sz}-bit public/private key pairs...")
    n, p, q = genPrivate(sz)
    phi = (p - 1) * (q - 1)
    fname = "poll_data.txt"
    
    if os.path.exists(fname): 
        os.remove(fname)

    num_voters = 25
    expected_yes_votes = 0
    print(f"\nSimulating {num_voters} voters...")

    for i in range(1, num_voters + 1):
        choice = random.choice([0, 1])
        
        if choice == 1:
            voteYes(fname, n)
            expected_yes_votes += 1
            # print(f"Voter {i}: YES") 
        else:
            voteNo(fname, n)
            # print(f"Voter {i}: NO") 

    print("-" * 30)
    print(f"Voting complete. Expected 'YES' count: {expected_yes_votes}")
    print("Processing encrypted ballots from file...")
    
    actual_tally = getResults(fname, n, phi)
    
    if actual_tally == expected_yes_votes:
        print("\nSUCCESS: The encrypted tally matches the manual count.")
    else:
        print("\nERROR: Mismatch detected between tally and manual count.")

--- Initializing Election System ---
Generating 128-bit public/private key pairs...

Simulating 25 voters...
------------------------------
Voting complete. Expected 'YES' count: 11
Processing encrypted ballots from file...
11

SUCCESS: The encrypted tally matches the manual count.
